In [1]:
# import Modules 
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('NFA 2019 public_data.csv')
df.head()

C:\Users\JK\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,Armenia,1992,1,AreaPerCap,0.140292,0.199546,0.097188051,0.036888,0.029320,0.000000,5.032351e-01,3A
1,Armenia,1992,1,AreaTotHA,483000.000000,687000.000000,334600,127000.000000,100943.000800,0.000000,1.732543e+06,3A
2,Armenia,1992,1,BiocapPerCap,0.159804,0.135261,0.084003213,0.013742,0.033398,0.000000,4.262086e-01,3A
3,Armenia,1992,1,BiocapTotGHA,550176.242700,465677.972200,289207.1078,47311.551720,114982.279300,0.000000,1.467355e+06,3A
4,Armenia,1992,1,EFConsPerCap,0.387510,0.189462,1.26E-06,0.004165,0.033398,1.114093,1.728629e+00,3A


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72186 entries, 0 to 72185
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country         72186 non-null  object 
 1   year            72186 non-null  int64  
 2   country_code    72186 non-null  int64  
 3   record          72186 non-null  object 
 4   crop_land       51714 non-null  float64
 5   grazing_land    51714 non-null  float64
 6   forest_land     51714 non-null  object 
 7   fishing_ground  51713 non-null  float64
 8   built_up_land   51713 non-null  float64
 9   carbon          51713 non-null  float64
 10  total           72177 non-null  float64
 11  QScore          72185 non-null  object 
dtypes: float64(6), int64(2), object(4)
memory usage: 6.6+ MB


In [4]:
df['QScore'].value_counts()

3A    51481
2A    10576
2B    10096
1A       16
1B       16
Name: QScore, dtype: int64

In [5]:
df.isna().sum()

country               0
year                  0
country_code          0
record                0
crop_land         20472
grazing_land      20472
forest_land       20472
fishing_ground    20473
built_up_land     20473
carbon            20473
total                 9
QScore                1
dtype: int64

In [6]:
df.dropna(inplace=True)

In [7]:
df.isna().sum()

country           0
year              0
country_code      0
record            0
crop_land         0
grazing_land      0
forest_land       0
fishing_ground    0
built_up_land     0
carbon            0
total             0
QScore            0
dtype: int64

In [8]:
df['QScore'].value_counts()

3A    51473
2A      224
1A       16
Name: QScore, dtype: int64

In [9]:
df['QScore'].replace(['1A'], '2A', inplace= True)

In [10]:
df['QScore'].value_counts()

3A    51473
2A      240
Name: QScore, dtype: int64

In [11]:
# Seperate each class
df_2A = df[df.QScore == '2A']

# get only 350 samples from 3A class
df_3A = df[df.QScore == '3A'].sample(350)
data_df = df_2A.append(df_3A)

## Preprocessing

In [12]:
import sklearn.utils
data_df = sklearn.utils.shuffle(data_df)
data_df.reset_index(drop= True, inplace= True)
data_df.shape , data_df.QScore.value_counts()

((590, 12),
 3A    350
 2A    240
 Name: QScore, dtype: int64)

In [13]:
data_df.drop(columns= ['country_code', 'country', 'year'], inplace= True)
X = data_df.drop(columns='QScore')
y = data_df['QScore']

In [14]:
#split the data into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= .3, random_state= 0)
y_train.value_counts()

3A    250
2A    163
Name: QScore, dtype: int64

In [15]:
#encode categorical variable
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
x_train.record = encoder.fit_transform(x_train.record)
x_test.record = encoder.transform(x_test.record)


In [16]:
import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1)
x_train_balanced, y_balanced = smote.fit_resample(x_train, y_train)

In [17]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_train_df = scaler.fit_transform(x_train_balanced.drop(columns=['record']))
normalised_train_df = pd.DataFrame(normalised_train_df, columns=x_train_balanced.drop(columns=['record']).columns)
normalised_train_df['record'] = x_train_balanced['record']

In [18]:
x_test = x_test.reset_index(drop=True)
normalised_test_df = scaler.transform(x_test.drop(columns=['record']))
normalised_test_df = pd.DataFrame(normalised_test_df, columns=x_test.drop(columns=['record']).columns)
normalised_test_df['record'] = x_test['record']

In [19]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(normalised_train_df, y_balanced)

LogisticRegression()

# Measuring Classification Performance

#### Accuracy, precision, recall, F1-score and many others are evaluation metrics used in measuring the performance of classification models.

In [20]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix

### Confusion Matrix

In [21]:
new_pred = log_reg.predict(normalised_test_df)
cnf_mat = confusion_matrix(y_true=y_test, y_pred=new_pred)
cnf_mat

array([[49, 28],
       [62, 38]], dtype=int64)

### Cross validation

In [22]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(log_reg, normalised_train_df, y_balanced, cv=5, scoring='f1_macro')
scores

array([0.6052232 , 0.43910256, 0.5       , 0.53964194, 0.61616162])

### Accuracy

In [23]:
accuracy = accuracy_score(y_true=y_test, y_pred=new_pred)
print('Accuracy: {}'.format(round(accuracy*100), 2))

Accuracy: 49


### Precision

In [24]:
precision = precision_score(y_true= y_test, y_pred=new_pred, pos_label='2A')
print('Precision: {}'.format(round(precision*100), 2))

Precision: 44


### Recall

In [25]:
recall = recall_score(y_true=y_test, y_pred=new_pred, pos_label='2A')
print('Recall: {}'.format(recall * 100), 2)

Recall: 63.63636363636363 2


### F1 Score

In [26]:
f1 = f1_score(y_true= y_test, y_pred=new_pred, pos_label='2A')
print('F1: {}'.format(f1 * 100), 2)

F1: 52.127659574468076 2


### K-Fold CV

In [27]:
from sklearn.model_selection import KFold
kf = KFold(n_splits= 5)
kf.split(normalised_train_df)
f1_scores = []

# Run for every split
for train_index , test_index  in kf.split(normalised_train_df):
    x_train, x_test = normalised_train_df.iloc[train_index],normalised_train_df.iloc[test_index]
    y_train, y_test = y_balanced[train_index], y_balanced[test_index]
    
    model = LogisticRegression().fit(x_train, y_train)
    # save result to list 
    f1_scores.append(f1_score(y_true=y_test, y_pred=model.predict(x_test), pos_label='2A')*100)
print(f1_scores)

[60.29411764705882, 51.85185185185186, 48.275862068965516, 51.72413793103448, 0.0]


### Stratified K-Fold

In [29]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle= True, random_state=1)
f1_scores = []

# Run for every split
for train_index, test_index in skf.split(normalised_train_df, y_balanced):
    x_train, x_test = np.array(normalised_train_df)[train_index], \
                            np.array(normalised_train_df)[test_index]
    y_train, y_test = y_balanced[train_index], y_balanced[test_index]
    
    model = LogisticRegression().fit(x_train, y_train)
    
    # save result to list
    f1_scores.append(f1_score(y_true=y_test, y_pred=model.predict(x_test), pos_label='2A'))
print(f1_scores)

[0.5510204081632654, 0.6153846153846153, 0.6153846153846154, 0.495049504950495, 0.5523809523809523]


###  Leave out one

In [32]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(LogisticRegression(), normalised_train_df, y_balanced, cv=loo,
                        scoring= 'f1_macro')
average_score = scores.mean() * 100
average_score

53.0